In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

np.random.seed(1337)
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000

# Data

In [2]:
# load the training data from the respective filepaths
train_features_filepath = 'data/train_features.csv'
test_features_filepath = 'data/test_features.csv'
train_labels_filepath = 'data/train_labels.csv'

# create the dataframes
train_features_df = pd.read_csv(train_features_filepath, float_precision="round_trip")
test_features_df = pd.read_csv(test_features_filepath, float_precision="round_trip")
train_labels_df = pd.read_csv(train_labels_filepath, float_precision="round_trip")
print("Train shape:", train_features_df.shape, "| Train label shape:", train_labels_df.shape, "| Test shape:", test_features_df.shape)
display(train_features_df.head(40))
display(train_labels_df.head(40))
display(test_features_df.head(30))

display(train_features_df.describe())
display(train_labels_df.describe())

Train shape: (227940, 37) | Train label shape: (18995, 16) | Test shape: (151968, 37)


,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,-2.0,16.0,NaN,NaN,6.3,NaN,45.0,NaN,NaN,NaN,NaN,NaN,84.0,1.2,3.8,61.0,NaN,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,4,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,-2.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,81.0,NaN,NaN,62.5,NaN,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,5,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,0.0,18.0,NaN,NaN,NaN,NaN,43.0,NaN,0.4,NaN,NaN,NaN,74.0,NaN,NaN,59.0,NaN,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,6,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,0.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,NaN,NaN,49.5,NaN,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,7,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,1.8,NaN,48.0,NaN,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
5,1,8,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4,NaN,NaN,NaN,68.0,1.8,NaN,51.0,NaN,NaN,100.0,NaN,NaN,22.4,82.0,NaN,NaN,106.0,NaN
6,1,9,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,NaN,NaN,46.0,NaN,NaN,100.0,NaN,NaN,NaN,67.0,NaN,NaN,112.0,NaN
7,1,10,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,NaN,NaN,47.0,NaN,NaN,100.0,NaN,NaN,NaN,62.0,NaN,NaN,121.0,NaN
8,1,11,34.0,NaN,NaN,12.0,NaN,NaN,8.5,26.0,NaN,12.0,NaN,4.6,4.7,0.50,NaN,NaN,NaN,143.0,NaN,120.0,67.0,2.1,4.1,47.0,7.60,NaN,100.0,NaN,111.0,23.8,58.0,NaN,NaN,118.0,NaN
9,1,12,34.0,NaN,NaN,12.0,NaN,38.0,8.5,26.0,0.0,18.0,NaN,NaN,4.7,NaN,42.0,NaN,0.4,NaN,NaN,NaN,62.0,2.1,4.1,44.0,NaN,NaN,100.0,NaN,111.0,23.8,66.0,NaN,NaN,110.0,7.39


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
2,100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
3,1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3
5,10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5,67.4,99.1,64.6
6,10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.2,84.9,96.8,90.5
7,10007,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,23.8,97.0,94.3,76.0
8,10009,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,18.9,72.0,95.3,91.5
9,1001,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14.9,83.3,99.9,117.3


,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,0,1,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2,39.0,NaN,44.2,17.0,NaN,36.0,10.2,13.0,NaN,NaN,147.0,6.0,17.5,2.2,NaN,32.0,0.60,194.0,NaN,273.0,77.0,2.2,4.60,76.0,8.00,119.0,100.0,NaN,98.0,31.0,82.0,21.8,NaN,119.0,NaN
2,0,3,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.0,13.0,NaN,NaN,NaN,NaN,26.0,NaN,0.55,NaN,NaN,NaN,78.0,NaN,NaN,72.5,NaN,NaN,100.0,NaN,NaN,NaN,78.0,NaN,NaN,125.0,7.34
3,0,4,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,0.50,NaN,NaN,NaN,87.0,NaN,NaN,66.0,NaN,NaN,100.0,NaN,NaN,NaN,80.0,NaN,NaN,136.0,NaN
4,0,5,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,NaN,NaN,65.0,NaN,NaN,100.0,NaN,NaN,NaN,83.0,NaN,NaN,135.0,NaN
5,0,6,39.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,NaN,NaN,66.0,NaN,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,144.0,NaN
6,0,7,39.0,NaN,38.5,20.0,NaN,NaN,9.1,16.0,NaN,12.0,NaN,4.8,18.5,2.4,NaN,31.0,0.50,193.0,NaN,162.0,NaN,2.5,3.70,NaN,8.30,109.0,100.0,NaN,102.0,25.9,NaN,26.4,NaN,NaN,NaN
7,0,8,39.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,-4.0,12.0,NaN,NaN,NaN,NaN,30.0,NaN,0.50,NaN,NaN,NaN,80.0,NaN,NaN,59.0,NaN,NaN,100.0,NaN,NaN,NaN,90.0,NaN,NaN,129.0,7.40
8,0,9,39.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,NaN,NaN,56.5,NaN,NaN,100.0,NaN,NaN,NaN,90.0,NaN,NaN,121.0,NaN
9,0,10,39.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,NaN,55.0,NaN,NaN,100.0,NaN,NaN,NaN,85.0,NaN,NaN,120.0,NaN


,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
count,227940.000000,227940.000000,227940.000000,9783.000000,10299.000000,20105.000000,10756.000000,81115.000000,22295.000000,12559.00000,19887.000000,187785.000000,2493.000000,11590.000000,19083.000000,17792.000000,21043.000000,5761.000000,26602.000000,18035.000000,13014.000000,47036.000000,195889.000000,17523.000000,28393.000000,152418.000000,17830.000000,5708.000000,195192.000000,719.000000,13917.000000,27297.000000,200128.000000,5326.000000,3776.000000,191650.000000,25046.000000
mean,15788.831219,7.014399,62.073809,32.883114,40.091310,23.192664,2.859716,36.852136,10.628208,23.48810,-1.239284,18.154043,262.496911,3.612519,11.738649,1.495777,41.115696,193.444888,0.701666,204.666426,93.010527,142.169407,82.117276,2.004149,4.152729,64.014711,7.161149,97.796163,97.663449,1.390723,106.260185,31.283090,84.522371,1.640941,7.269240,122.369877,7.367231
std,9151.896286,4.716103,16.451854,7.802065,26.034961,20.024289,2.428368,0.875152,2.074859,4.40378,4.192677,5.037031,133.020910,1.384462,10.088872,1.898112,8.929873,682.836708,24.522126,104.156406,10.887271,56.894530,16.471871,0.437286,0.670168,13.920097,2.812067,122.773379,2.786186,2.792722,5.916082,5.770425,17.643437,3.244145,25.172442,23.273834,0.074384
min,1.000000,1.000000,15.000000,10.000000,12.500000,1.000000,0.200000,21.000000,3.300000,0.00000,-29.000000,1.000000,34.000000,0.200000,0.100000,0.100000,10.000000,5.000000,0.000000,2.000000,24.000000,15.000000,20.000000,0.500000,1.300000,20.000000,1.000000,12.000000,20.000000,0.010000,66.000000,9.400000,23.000000,0.100000,0.010000,21.000000,6.820000
25%,7879.000000,4.000000,52.000000,28.500000,27.800000,12.000000,1.400000,36.000000,9.200000,21.00000,-3.000000,15.000000,177.000000,2.800000,7.600000,0.700000,36.000000,21.000000,0.400000,137.000000,95.000000,108.000000,71.000000,1.700000,3.700000,54.250000,7.300000,53.000000,96.000000,0.100000,103.000000,27.300000,72.000000,0.500000,0.030000,105.000000,7.330000
50%,15726.000000,7.000000,64.000000,33.000000,32.200000,17.000000,2.100000,37.000000,10.500000,23.90000,-1.000000,18.000000,233.000000,3.400000,10.400000,0.900000,40.000000,36.000000,0.500000,189.000000,97.000000,130.000000,80.000000,2.000000,4.100000,62.000000,8.200000,72.000000,98.000000,0.300000,107.000000,30.900000,83.000000,0.800000,0.150000,119.000000,7.370000
75%,23725.000000,10.000000,74.000000,38.000000,40.600000,27.000000,3.400000,37.000000,12.000000,26.00000,0.000000,21.000000,316.000000,4.200000,14.000000,1.380000,45.000000,84.000000,0.600000,251.000000,98.000000,160.000000,91.000000,2.200000,4.500000,72.000000,8.700000,104.000000,100.000000,1.210000,110.000000,35.000000,95.000000,1.400000,2.050000,137.000000,7.410000
max,31658.000000,315.000000,100.000000,100.000000,250.000000,268.000000,31.000000,42.000000,23.800000,50.00000,100.000000,97.000000,1179.000000,16.400000,440.000000,41.900000,100.000000,9961.000000,4000.000000,2322.000000,100.000000,952.000000,300.000000,9.600000,10.750000,298.000000,20.600000,3833.000000,100.000000,21.200000,141.000000,63.400000,191.000000,46.500000,440.000000,287.000000,7.780000


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
count,18995.000000,18995.000000,18995.000000,18995.000000,18995.00000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000
mean,15788.831219,0.268281,0.073704,0.239747,0.23622,0.240590,0.200211,0.099763,0.233693,0.033904,0.066017,0.057278,18.795960,82.511171,96.947311,84.119716
std,9152.117122,0.443076,0.261295,0.426940,0.42477,0.427453,0.400168,0.299692,0.423190,0.180986,0.248319,0.232380,3.511241,12.745110,2.110957,14.718396
min,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,26.000000,27.000000,30.200000
25%,7879.500000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.550000,73.200000,95.900000,73.700000
50%,15726.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.400000,81.000000,97.100000,83.400000
75%,23724.500000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.600000,90.200000,98.300000,93.600000
max,31658.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,41.100000,147.100000,100.000000,155.600000


In [3]:
labels_sepsis_positive = train_labels_df.loc[train_labels_df['LABEL_Sepsis'] == 1]
display(labels_sepsis_positive.describe())
display(labels_sepsis_positive.head(100))

#temp_grouped = group_columns_in_df(train_features_df)
#display(temp_grouped.head(10))
#features_sepsis_positive = temp_grouped.loc[temp_grouped['pid'].isin(labels_sepsis_positive['pid'])]
#scale_df(features_sepsis_positive)
#display(features_sepsis_positive.describe())

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
count,1088.000000,1088.000000,1088.000000,1088.000000,1088.000000,1088.000000,1088.000000,1088.000000,1088.000000,1088.000000,1088.000000,1088.0,1088.000000,1088.000000,1088.000000,1088.000000
mean,15935.926471,0.547794,0.177390,0.465993,0.460478,0.469669,0.605699,0.129596,0.511949,0.100184,0.189338,1.0,19.881541,80.647059,97.036397,88.546599
std,9166.983876,0.497939,0.382174,0.499072,0.498665,0.499309,0.488925,0.336012,0.500087,0.300383,0.391957,0.0,4.631534,11.236957,2.451813,14.966026
min,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,51.800000,67.000000,49.800000
25%,8043.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,16.700000,72.800000,95.900000,78.175000
50%,16180.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.0,19.500000,79.550000,97.400000,87.800000
75%,23941.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.0,22.500000,88.000000,98.600000,98.325000
max,31515.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,36.900000,125.300000,100.000000,134.700000


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
7,10007,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,23.8,97.0,94.3,76.0
20,10023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,13.3,80.3,96.7,78.0
25,10030,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,17.9,85.3,99.8,95.2
27,10034,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,16.1,89.2,99.7,65.4
31,10038,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,18.4,62.0,97.1,81.1
35,10048,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,18.4,95.5,97.6,98.6
53,10072,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,15.7,78.8,97.6,91.7
83,10116,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,15.8,91.2,99.3,74.9
93,10130,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,20.6,78.2,97.0,101.8
109,10150,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,18.1,72.8,97.9,67.0


## Grouping

* groupby the `'pid'` column values for each age range from `[0, 20, 30, 40, 50, 60, 70, 80, 110]`
* ignore `'Time'` col

In [4]:
def group_columns_in_df(df):
    age_ranges = [0, 20, 30, 40, 50, 60, 70, 80, 110]
    dfs = []
    for i in range(len(age_ranges) - 1):
        age_l = age_ranges[i]
        age_u = age_ranges[i+1]
        sub_df = df[(df['Age'] >= age_l) & (df['Age'] < age_u)]
        sub_df = sub_df.loc[:, sub_df.columns != 'Time'].groupby('pid', as_index=False, sort=False).agg(np.nanmean)
        sub_df = sub_df.fillna(sub_df.mean())
        dfs.append(sub_df)
    grouped_df = df.loc[:, df.columns != 'Time'].groupby('pid', as_index=False, sort=False).agg(np.nanmean)
    result = grouped_df.copy()
    result.update(result[['pid']].merge(pd.concat(dfs)))
    result = result.astype({"pid": int})
    assert(grouped_df.shape == result.shape)
    assert(not grouped_df.equals(result))
    return result

X_df = group_columns_in_df(train_features_df)
display(X_df.shape)
display(X_df.head(20))

X_test_df = group_columns_in_df(test_features_df)

(18995, 36)

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,34.0,35.270246,36.296005,12.000000,2.267844,36.750000,8.566667,25.333333,-0.666667,17.000000,282.406566,4.600000,5.233333,0.500000,43.333333,179.917422,0.425000,143.000000,95.156566,120.000000,68.333333,1.800000,4.000000,50.250000,7.600000,101.242051,100.000000,1.973750,112.000000,23.200000,77.083333,2.196677,3.712206,114.500000,7.370000
1,10,71.0,31.424716,27.800000,12.000000,2.395573,36.000000,14.600000,23.828877,-0.881882,18.090909,274.985885,2.500000,11.500000,0.820000,40.970572,20.000000,0.552612,207.000000,92.402792,152.000000,101.727273,1.500000,3.200000,83.272727,8.600000,68.000000,98.000000,0.608505,106.167733,42.100000,78.818182,1.300000,0.010000,132.909091,7.375165
2,100,68.0,32.960816,20.900000,21.000000,2.376910,36.250000,12.500000,27.000000,-0.727915,14.833333,270.562388,3.500000,12.500000,1.100000,41.495807,146.018250,1.069466,204.000000,93.827695,243.000000,81.833333,1.700000,3.600000,62.833333,9.000000,90.714723,96.500000,1.046905,101.000000,36.800000,109.083333,1.435074,6.996616,117.000000,7.372458
3,1000,79.0,31.863636,39.371346,22.000000,3.855000,36.818182,9.200000,23.828877,-0.881882,12.000000,274.985885,1.900000,19.600000,0.960000,44.000000,86.038020,0.400000,158.000000,98.000000,128.625000,83.454545,2.000000,3.966667,62.818182,3.463333,97.562013,98.818182,0.608505,106.167733,27.300000,86.363636,1.197754,5.817339,141.909091,7.300000
4,10000,76.0,31.424716,28.550000,22.000000,2.395573,36.750000,10.700000,25.500000,1.500000,12.090909,274.985885,3.609838,7.750000,1.000000,44.500000,86.038020,0.500000,135.000000,98.250000,121.750000,69.090909,1.400000,3.900000,48.227273,7.705750,97.562013,98.545455,0.608505,103.500000,30.300000,77.090909,1.197754,5.817339,123.000000,7.390000
5,10002,73.0,19.000000,31.300000,18.000000,3.005000,37.000000,10.400000,23.828877,-0.881882,19.625000,161.000000,3.000000,10.300000,0.980000,40.500000,41.000000,0.800000,83.000000,92.402792,127.166667,69.818182,2.100000,4.475000,48.500000,3.120000,38.000000,99.181818,0.608505,107.000000,30.300000,67.090909,0.800000,5.817339,132.090909,7.375000
6,10006,51.0,34.020707,37.275808,21.317991,2.395311,37.500000,10.949367,23.963837,-0.580284,18.888889,268.857981,3.661243,11.297389,1.438611,41.414587,176.596782,0.538963,212.857003,94.235683,200.500000,70.555556,1.987571,4.112962,48.714286,7.742205,98.531877,96.666667,1.586346,105.757518,32.484703,82.000000,1.816378,7.683001,117.888889,7.374335
7,10007,60.0,32.960816,39.756291,23.358213,2.376910,38.000000,10.672379,24.035953,-0.727915,21.909091,270.562388,2.400000,11.619513,1.500620,41.495807,146.018250,1.069466,209.292544,93.827695,87.000000,108.181818,1.600000,3.600000,88.363636,7.524813,90.714723,94.909091,1.046905,105.700469,31.858225,79.909091,1.435074,0.080000,139.363636,7.372458
8,10009,69.0,32.960816,86.050000,15.000000,2.376910,37.250000,12.200000,21.000000,-0.727915,22.500000,270.562388,3.100000,8.700000,0.500000,41.495807,17.000000,1.069466,182.000000,93.827695,109.000000,65.909091,1.200000,4.100000,65.227273,8.100000,64.000000,97.090909,1.046905,104.000000,34.000000,97.727273,0.800000,6.996616,90.909091,7.372458
9,1001,36.0,35.270246,31.200000,10.000000,1.800000,37.666667,10.400000,31.000000,7.500000,13.363636,282.406566,2.100000,8.700000,0.500000,40.250000,179.917422,0.416667,205.000000,95.156566,98.666667,85.909091,2.000000,3.100000,69.181818,8.500000,101.242051,100.000000,1.973750,105.000000,29.800000,106.727273,2.196677,3.712206,113.090909,7.515000


## Adding features

In [5]:
def max_abs_diff_agg_fn(x):
    if x.isna().all():
        return 0
    else:
        min_val = np.nanmin(x)
        max_val = np.nanmax(x)
        res = np.abs(max_val - min_val)
        return res

def std_dev_agg_fn(x):
    if x.isna().all():
        return 0
    else:
        return np.nanstd(x)

def add_features(df_to_group, df_to_add_features_to):
    """Add maximum absolute difference features for each column to a dataframe
    
    Args:
        df_to_group (pd.DataFrame): the dataframe that will be grouped by 'pid'
        df_to_add_features_to (pd.DataFrame): the dataframe to add new features to
        
    Returns:
        pd.DataFrame: the new dataframe with all features
    """
    ## Maximum Absolute Difference
    mad_features_df = df_to_group.drop(['Age', 'Time'], axis=1).groupby('pid', as_index=False, sort=False).agg(max_abs_diff_agg_fn)
    mad_features_df.columns = [str(col) + '_diff' for col in mad_features_df.columns]
    # Remove 'pid' col from features_df
    mad_features_df = mad_features_df.iloc[:, 1:]
    
    ## Std dev.
    std_features_df = df_to_group.drop(['Age', 'Time'], axis=1).groupby('pid', as_index=False, sort=False).agg(std_dev_agg_fn)
    std_features_df.columns = [str(col) + '_std' for col in std_features_df.columns]
    # Remove 'pid' col from features_df
    std_features_df = std_features_df.iloc[:, 1:]
    
    ## Result
    return pd.concat([df_to_add_features_to, mad_features_df, std_features_df], axis=1, sort=False)

X_df = add_features(train_features_df, X_df)

display(X_df.shape)
display(X_df.describe())
display(X_df.head(30))


X_test_df = add_features(test_features_df, X_test_df)

(18995, 70)

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH,EtCO2_diff,PTT_diff,BUN_diff,Lactate_diff,Temp_diff,Hgb_diff,HCO3_diff,BaseExcess_diff,RRate_diff,Fibrinogen_diff,Phosphate_diff,WBC_diff,Creatinine_diff,PaCO2_diff,AST_diff,FiO2_diff,Platelets_diff,SaO2_diff,Glucose_diff,ABPm_diff,Magnesium_diff,Potassium_diff,ABPd_diff,Calcium_diff,Alkalinephos_diff,SpO2_diff,Bilirubin_direct_diff,Chloride_diff,Hct_diff,Heartrate_diff,Bilirubin_total_diff,TroponinI_diff,ABPs_diff,pH_diff,EtCO2_std,PTT_std,BUN_std,Lactate_std,Temp_std,Hgb_std,HCO3_std,BaseExcess_std,RRate_std,Fibrinogen_std,Phosphate_std,WBC_std,Creatinine_std,PaCO2_std,AST_std,FiO2_std,Platelets_std,SaO2_std,Glucose_std,ABPm_std,Magnesium_std,Potassium_std,ABPd_std,Calcium_std,Alkalinephos_std,SpO2_std,Bilirubin_direct_std,Chloride_std,Hct_std,Heartrate_std,Bilirubin_total_std,TroponinI_std,ABPs_std,pH_std
count,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.00000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000
mean,15788.831219,62.073809,32.573714,38.715634,23.013123,2.381693,36.773611,10.753563,23.794300,-0.799614,18.126792,272.512791,3.619892,11.562789,1.469609,40.899609,152.631039,0.665865,210.803602,93.423906,138.704813,82.395581,1.995613,4.112740,64.339239,7.73675,96.374003,97.609488,1.036307,105.925660,32.017304,84.337887,1.495649,6.311410,122.605635,7.375433,0.528429,1.615112,0.514872,0.174562,0.805054,0.282035,0.265041,0.683906,8.720347,1.633788,0.074775,0.482395,0.034124,2.395322,4.685601,0.289954,5.345854,1.551198,29.346144,27.566886,0.064127,0.209895,17.567287,0.488873,0.401027,4.435588,0.002144,0.370045,1.164927,20.124401,0.014262,0.162460,37.655067,0.022964,0.178832,0.773258,0.248603,0.073066,0.336823,0.128371,0.125337,0.274006,2.691511,0.764850,0.036204,0.230158,0.016587,0.962879,2.276308,0.123756,2.553527,0.654652,11.733214,8.219497,0.031062,0.092110,5.287030,0.227511,0.194112,1.415197,0.001032,0.173813,0.511944,6.222233,0.006976,0.079508,11.368657,0.009109
std,9152.117122,16.452251,2.734075,15.321136,17.064693,0.932292,0.662711,1.772299,2.720111,2.224652,3.991447,42.200867,0.918713,7.641441,1.611854,5.523614,308.154375,7.256601,87.271626,4.765965,44.638819,13.550943,0.314889,0.532058,10.560827,1.61900,55.360941,2.226629,0.678024,3.786477,5.099023,16.132903,1.584337,9.162408,19.250298,0.042436,2.478752,10.088362,2.190143,0.758831,0.859484,0.766081,1.031707,1.914602,5.241411,15.657243,0.335666,2.260060,0.209410,5.357459,98.659798,29.019941,17.674116,6.482434,51.725303,15.758737,0.237366,0.452224,14.878999,1.703414,7.829698,

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH,EtCO2_diff,PTT_diff,BUN_diff,Lactate_diff,Temp_diff,Hgb_diff,HCO3_diff,BaseExcess_diff,RRate_diff,Fibrinogen_diff,Phosphate_diff,WBC_diff,Creatinine_diff,PaCO2_diff,AST_diff,FiO2_diff,Platelets_diff,SaO2_diff,Glucose_diff,ABPm_diff,Magnesium_diff,Potassium_diff,ABPd_diff,Calcium_diff,Alkalinephos_diff,SpO2_diff,Bilirubin_direct_diff,Chloride_diff,Hct_diff,Heartrate_diff,Bilirubin_total_diff,TroponinI_diff,ABPs_diff,pH_diff,EtCO2_std,PTT_std,BUN_std,Lactate_std,Temp_std,Hgb_std,HCO3_std,BaseExcess_std,RRate_std,Fibrinogen_std,Phosphate_std,WBC_std,Creatinine_std,PaCO2_std,AST_std,FiO2_std,Platelets_std,SaO2_std,Glucose_std,ABPm_std,Magnesium_std,Potassium_std,ABPd_std,Calcium_std,Alkalinephos_std,SpO2_std,Bilirubin_direct_std,Chloride_std,Hct_std,Heartrate_std,Bilirubin_total_std,TroponinI_std,ABPs_std,pH_std
0,1,34.0,35.270246,36.296005,12.000000,2.267844,36.750000,8.566667,25.333333,-0.666667,17.000000,282.406566,4.600000,5.233333,0.500000,43.333333,179.917422,0.425000,143.000000,95.156566,120.000000,68.333333,1.800000,4.000000,50.250000,7.600000,101.242051,100.000000,1.973750,112.000000,23.200000,77.083333,2.196677,3.712206,114.500000,7.370000,0.0,0.0,0.0,0.00,2.0,0.2,2.0,2.0,6.0,0.0,0.0,1.6,0.00,3.0,0.0,0.1,0.0,0.0,0.0,31.0,0.9,0.30,23.50,0.00,0.0,0.0,0.0,3.0,2.4,41.0,0.0,0.00,42.0,0.08,0.000000,0.00,0.000000,0.000,0.661438,0.094281,0.942809,0.942809,1.732051,0.0,0.00000,0.754247,0.00000,1.247219,0.0,4.330127e-02,0.0,0.000000,0.000000,7.961714,0.328634,0.141421,6.799203,0.000000,0.0,0.000000,0.0,1.414214,0.909212,13.325779,0.0,0.000,11.572237,0.028284
1,10,71.0,31.424716,27.800000,12.000000,2.395573,36.000000,14.600000,23.828877,-0.881882,18.090909,274.985885,2.500000,11.500000,0.820000,40.970572,20.000000,0.552612,207.000000,92.402792,152.000000,101.727273,1.500000,3.200000,83.272727,8.600000,68.000000,98.000000,0.608505,106.167733,42.100000,78.818182,1.300000,0.010000,132.909091,7.375165,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0,0.00,35.00,0.00,0.0,3.0,0.0,0.0,0.0,19.0,0.0,0.00,39.0,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,3.028787,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.000000,10.109320,0.000000,0.000000,10.136262,0.000000,0.0,1.044466,0.0,0.000000,0.000000,6.235065,0.0,0.000,11.508531,0.000000
2,100,68.0,32.960816,20.900000,21.000000,2.376910,36.250000,12.500000,27.000000,-0.727915,14.833333,270.562388,3.500000,12.500000,1.100000,41.495807,146.018250,1.069466,204.000000,93.827695,243.000000,81.833333,1.700000,3.600000,62.833333,9.000000,90.714723,96.500000,1.046905,101.000000,36.800000,109.083333,1.435074,6.996616,117.000000,7.372458,0.0,0.0,0.0,0.00,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.00,20.00,0.00,0.0,8.0,0.0,0.0,0.0,37.0,0.0,0.00,61.0,0.00,0.000000,0.00,0.000000,0.000,1.089725,0.000000,0.000000,0.000000,1.624466,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.000000,8.060535,0.000000,0.000000,5.288877,0.000000,0.0,2.254625,0.0,0.000000,0.000000,12.086207,0.0,0.000,15.538808,0.000000
3,1000,79.0,31.863636,39.371346,22.000000,3.855000,36.818182,9.200000,23.828877,-0.881882,12.000000,274.985885,1.900000,19.600000,0.960000,44.000000,86.038020,0.400000,158.000000,98.000000,128.625000,83.454545,2.000000,3.966667,62.818182,3.463333,97.562013,98.818182,0.608505,106.167733,27.300000,86.363636,1.197754,5.817339,141.909091,7.300000,11.0,0.0,0.0,0.39,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,100.5,30.0,0.0,0.60,14.00,6.98,0.0,5.0,0.0,0.0,0.0,41.0,0.0,0.00,67.0,0.00,3.643743,0.00,0.000000,0.195,0.833196,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000e+00,0.

## Mean value imputation

In [6]:
def mean_impute_df(df):
    return df.fillna(df.mean())

X_df = mean_impute_df(X_df)
display(X_df.head(30))

X_test_df = mean_impute_df(X_test_df)

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH,EtCO2_diff,PTT_diff,BUN_diff,Lactate_diff,Temp_diff,Hgb_diff,HCO3_diff,BaseExcess_diff,RRate_diff,Fibrinogen_diff,Phosphate_diff,WBC_diff,Creatinine_diff,PaCO2_diff,AST_diff,FiO2_diff,Platelets_diff,SaO2_diff,Glucose_diff,ABPm_diff,Magnesium_diff,Potassium_diff,ABPd_diff,Calcium_diff,Alkalinephos_diff,SpO2_diff,Bilirubin_direct_diff,Chloride_diff,Hct_diff,Heartrate_diff,Bilirubin_total_diff,TroponinI_diff,ABPs_diff,pH_diff,EtCO2_std,PTT_std,BUN_std,Lactate_std,Temp_std,Hgb_std,HCO3_std,BaseExcess_std,RRate_std,Fibrinogen_std,Phosphate_std,WBC_std,Creatinine_std,PaCO2_std,AST_std,FiO2_std,Platelets_std,SaO2_std,Glucose_std,ABPm_std,Magnesium_std,Potassium_std,ABPd_std,Calcium_std,Alkalinephos_std,SpO2_std,Bilirubin_direct_std,Chloride_std,Hct_std,Heartrate_std,Bilirubin_total_std,TroponinI_std,ABPs_std,pH_std
0,1,34.0,35.270246,36.296005,12.000000,2.267844,36.750000,8.566667,25.333333,-0.666667,17.000000,282.406566,4.600000,5.233333,0.500000,43.333333,179.917422,0.425000,143.000000,95.156566,120.000000,68.333333,1.800000,4.000000,50.250000,7.600000,101.242051,100.000000,1.973750,112.000000,23.200000,77.083333,2.196677,3.712206,114.500000,7.370000,0.0,0.0,0.0,0.00,2.0,0.2,2.0,2.0,6.0,0.0,0.0,1.6,0.00,3.0,0.0,0.1,0.0,0.0,0.0,31.0,0.9,0.30,23.50,0.00,0.0,0.0,0.0,3.0,2.4,41.0,0.0,0.00,42.0,0.08,0.000000,0.00,0.000000,0.000,0.661438,0.094281,0.942809,0.942809,1.732051,0.0,0.00000,0.754247,0.00000,1.247219,0.0,4.330127e-02,0.0,0.000000,0.000000,7.961714,0.328634,0.141421,6.799203,0.000000,0.0,0.000000,0.0,1.414214,0.909212,13.325779,0.0,0.000,11.572237,0.028284
1,10,71.0,31.424716,27.800000,12.000000,2.395573,36.000000,14.600000,23.828877,-0.881882,18.090909,274.985885,2.500000,11.500000,0.820000,40.970572,20.000000,0.552612,207.000000,92.402792,152.000000,101.727273,1.500000,3.200000,83.272727,8.600000,68.000000,98.000000,0.608505,106.167733,42.100000,78.818182,1.300000,0.010000,132.909091,7.375165,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0,0.00,35.00,0.00,0.0,3.0,0.0,0.0,0.0,19.0,0.0,0.00,39.0,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,3.028787,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.000000,10.109320,0.000000,0.000000,10.136262,0.000000,0.0,1.044466,0.0,0.000000,0.000000,6.235065,0.0,0.000,11.508531,0.000000
2,100,68.0,32.960816,20.900000,21.000000,2.376910,36.250000,12.500000,27.000000,-0.727915,14.833333,270.562388,3.500000,12.500000,1.100000,41.495807,146.018250,1.069466,204.000000,93.827695,243.000000,81.833333,1.700000,3.600000,62.833333,9.000000,90.714723,96.500000,1.046905,101.000000,36.800000,109.083333,1.435074,6.996616,117.000000,7.372458,0.0,0.0,0.0,0.00,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.00,20.00,0.00,0.0,8.0,0.0,0.0,0.0,37.0,0.0,0.00,61.0,0.00,0.000000,0.00,0.000000,0.000,1.089725,0.000000,0.000000,0.000000,1.624466,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.000000,8.060535,0.000000,0.000000,5.288877,0.000000,0.0,2.254625,0.0,0.000000,0.000000,12.086207,0.0,0.000,15.538808,0.000000
3,1000,79.0,31.863636,39.371346,22.000000,3.855000,36.818182,9.200000,23.828877,-0.881882,12.000000,274.985885,1.900000,19.600000,0.960000,44.000000,86.038020,0.400000,158.000000,98.000000,128.625000,83.454545,2.000000,3.966667,62.818182,3.463333,97.562013,98.818182,0.608505,106.167733,27.300000,86.363636,1.197754,5.817339,141.909091,7.300000,11.0,0.0,0.0,0.39,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,100.5,30.0,0.0,0.60,14.00,6.98,0.0,5.0,0.0,0.0,0.0,41.0,0.0,0.00,67.0,0.00,3.643743,0.00,0.000000,0.195,0.833196,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000e+00,0.

## Scaling / normalization

In [7]:
from sklearn import preprocessing

# Scale the dataframe
def scale_df(df):
    scaler = preprocessing.MinMaxScaler()
    df.loc[:, df.columns != 'pid'] = scaler.fit_transform(df.loc[:, df.columns != 'pid'])

scale_df(X_df)
display(X_df.describe())
display(X_df.head(30))

scale_df(X_test_df)

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH,EtCO2_diff,PTT_diff,BUN_diff,Lactate_diff,Temp_diff,Hgb_diff,HCO3_diff,BaseExcess_diff,RRate_diff,Fibrinogen_diff,Phosphate_diff,WBC_diff,Creatinine_diff,PaCO2_diff,AST_diff,FiO2_diff,Platelets_diff,SaO2_diff,Glucose_diff,ABPm_diff,Magnesium_diff,Potassium_diff,ABPd_diff,Calcium_diff,Alkalinephos_diff,SpO2_diff,Bilirubin_direct_diff,Chloride_diff,Hct_diff,Heartrate_diff,Bilirubin_total_diff,TroponinI_diff,ABPs_diff,pH_diff,EtCO2_std,PTT_std,BUN_std,Lactate_std,Temp_std,Hgb_std,HCO3_std,BaseExcess_std,RRate_std,Fibrinogen_std,Phosphate_std,WBC_std,Creatinine_std,PaCO2_std,AST_std,FiO2_std,Platelets_std,SaO2_std,Glucose_std,ABPm_std,Magnesium_std,Potassium_std,ABPd_std,Calcium_std,Alkalinephos_std,SpO2_std,Bilirubin_direct_std,Chloride_std,Hct_std,Heartrate_std,Bilirubin_total_std,TroponinI_std,ABPs_std,pH_std
count,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.00000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000,18995.000000
mean,15788.831219,0.553810,0.250819,0.093589,0.082446,0.088835,0.492125,0.360469,0.397570,0.474517,0.179338,0.228158,0.202904,0.029627,0.032766,0.355168,0.015212,0.000666,0.089216,0.898829,0.154757,0.366054,0.233690,0.245766,0.265064,0.343712,0.022082,0.967253,0.048434,0.470037,0.408432,0.422478,0.030079,0.014322,0.432140,0.621882,0.013549,0.007081,0.006436,0.006325,0.050316,0.025875,0.009139,0.006332,0.126382,0.002453,0.008402,0.003951,0.004303,0.036293,0.000744,0.000072,0.010421,0.023865,0.035187,0.118823,0.007205,0.033799,0.071123,0.026326,0.000593,0.056867,0.000523,0.009738,0.037457,0.159717,0.002161,0.000874,0.212740,0.038274,0.011727,0.006780,0.006215,0.005717,0.065741,0.023554,0.009325,0.006003,0.138069,0.002297,0.008136,0.003770,0.004183,0.035014,0.000723,0.000071,0.011054,0.021118,0.028137,0.117821,0.00900,0.033494,0.072380,0.029857,0.000574,0.054431,0.000504,0.009148,0.032922,0.118417,0.002114,0.000855,0.210940,0.035906
std,9152.117122,0.193556,0.030379,0.065728,0.063913,0.039785,0.068321,0.086877,0.062531,0.043621,0.041795,0.040539,0.057300,0.019750,0.038561,0.063490,0.031752,0.007254,0.037650,0.073323,0.057710,0.094930,0.049201,0.060461,0.075645,0.082602,0.014489,0.030502,0.031997,0.059473,0.096117,0.118308,0.034145,0.020824,0.116176,0.053267,0.063558,0.044228,0.027377,0.027494,0.053718,0.070283,0.035576,0.017728,0.075962,0.023509,0.037715,0.018510,0.026407,0.081174,0.015670,0.007256,0.034452,0.099730,0.062021,0.067926,0.026670,0.072822,0.060239,0.091729,0.011582,0.054560,0.016270,0.036057,0.082283,0.094252,0.023338,0.0

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH,EtCO2_diff,PTT_diff,BUN_diff,Lactate_diff,Temp_diff,Hgb_diff,HCO3_diff,BaseExcess_diff,RRate_diff,Fibrinogen_diff,Phosphate_diff,WBC_diff,Creatinine_diff,PaCO2_diff,AST_diff,FiO2_diff,Platelets_diff,SaO2_diff,Glucose_diff,ABPm_diff,Magnesium_diff,Potassium_diff,ABPd_diff,Calcium_diff,Alkalinephos_diff,SpO2_diff,Bilirubin_direct_diff,Chloride_diff,Hct_diff,Heartrate_diff,Bilirubin_total_diff,TroponinI_diff,ABPs_diff,pH_diff,EtCO2_std,PTT_std,BUN_std,Lactate_std,Temp_std,Hgb_std,HCO3_std,BaseExcess_std,RRate_std,Fibrinogen_std,Phosphate_std,WBC_std,Creatinine_std,PaCO2_std,AST_std,FiO2_std,Platelets_std,SaO2_std,Glucose_std,ABPm_std,Magnesium_std,Potassium_std,ABPd_std,Calcium_std,Alkalinephos_std,SpO2_std,Bilirubin_direct_std,Chloride_std,Hct_std,Heartrate_std,Bilirubin_total_std,TroponinI_std,ABPs_std,pH_std
0,1,0.223529,0.280781,0.083209,0.041199,0.083976,0.489691,0.253268,0.432950,0.477124,0.167539,0.237662,0.264033,0.013268,0.009569,0.383142,0.018023,0.000425,0.059965,0.925486,0.130575,0.267541,0.203125,0.232955,0.164146,0.336735,0.023356,1.000000,0.092673,0.565445,0.242224,0.369278,0.045187,0.008414,0.383223,0.615063,0.000000,0.000000,0.000,0.000000,0.1250,0.018349,0.068966,0.018519,0.086957,0.0,0.000000,0.013104,0.000000,0.045455,0.000000,0.000025,0.000000,0.000000,0.000000,0.133621,0.101124,0.048309,0.095142,0.000000,0.000000,0.000000,0.0,0.078947,0.077170,0.325397,0.000000,0.000000,0.237288,0.133333,0.000000,0.000000,0.000000,0.000000,0.129099,0.017299,0.070143,0.020655,0.088850,0.0,0.000000,0.012355,0.000000,0.045353,0.000000,2.500250e-05,0.000000,0.000000,0.000000,0.114125,0.095225,0.051426,0.093082,0.000000,0.000000,0.000000,0.0,0.074432,0.058470,0.253606,0.000000,0.000000,0.214718,0.111494
1,10,0.658824,0.238052,0.046761,0.041199,0.089427,0.412371,0.549020,0.398365,0.472904,0.178962,0.230534,0.133056,0.029465,0.017225,0.355984,0.001546,0.000552,0.087575,0.883120,0.171946,0.501481,0.156250,0.142045,0.400680,0.387755,0.014656,0.972603,0.028245,0.473839,0.598492,0.382000,0.025862,0.000000,0.494322,0.621546,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.000000,0.115942,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.159483,0.000000,0.000000,0.141700,0.000000,0.000000,0.038462,0.0,0.000000,0.000000,0.150794,0.000000,0.000000,0.220339,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.155370,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.144910,0.000000,0.000000,0.138766,0.000000,0.000000,0.040172,0.0,0.000000,0.000000,0.118661,0.000000,0.000000,0.213536,0.000000
2,100,0.623529,0.255120,0.017160,0.074906,0.088631,0.438144,0.446078,0.471264,0.475923,0.144852,0.226285,0.195426,0.032050,0.023923,0.362021,0.014530,0.001069,0.086281,0.905041,0.289593,0.362115,0.187500,0.187500,0.254278,0.408163,0.020601,0.952055,0.048934,0.392670,0.498586,0.603944,0.028773,0.015879,0.398310,0.618148,0.000000,0.000000,0.000,0.000000,0.1875,0.000000,0.000000,0.000000,0.072464,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.137931,0.000000,0.000000,0.080972,0.000000,0.000000,0.102564,0.0,0.000000,0.000000,0.293651,0.000000,0.000000,0.344633,0.000000,0.000000,0.000000,0.000000,0.000000,0.212692,0.000000,0.000000,0.000000,0.083332,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.115542,0.000000,0.000000,0.072405,0.000000,0.000000,0.086716,0.0,0.000000,0.000000,0.230015,0.000000,0.000000,0.288315,0.000000
3,1000,0.752941,0.242929,0.096402,0.078652,0.151707,0.496720,0.284314,0.398365,0.472904,0.115183,0.230534,0.095634,0.050401,0.020574,0.390805,0.008350,0.000400,0.066437,0.969231,0.1

# SVM Training

In [8]:
#X_df = X_df.iloc[0:2000, :]
#train_labels_df = train_labels_df.iloc[0:2000, :]

# Prepare train set
# Assert that the pids are matching in train features / train labels dfs
assert(X_df.iloc[:, 0].astype(int).equals(train_labels_df.iloc[:, 0].astype(int)))

X = X_df.iloc[:, 1:].to_numpy()
y = train_labels_df.iloc[:, 1:].to_numpy()

assert(X.shape[0] == y.shape[0])

In [9]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

model = MultiOutputRegressor(SVR(verbose=True, C=100, epsilon=0.1), n_jobs=-1)

model.fit(X, y)

MultiOutputRegressor(estimator=SVR(C=100, cache_size=200, coef0=0.0, degree=3,
                                   epsilon=0.1, gamma='scale', kernel='rbf',
                                   max_iter=-1, shrinking=True, tol=0.001,
                                   verbose=True),
                     n_jobs=-1)

# Predictions

In [10]:
def predict(df):
    # Create X_predict by removing 'pid' column
    X_predict = df.iloc[:, 1:].to_numpy()
    
    # Predict
    predictions = model.predict(X_predict)
    predictions[:, :-4] = np.divide(1, 1+np.exp(-predictions[:, :-4]))
    
    # Create predictions df
    predict_labels_df = pd.DataFrame(columns=train_labels_df.columns)
    predict_labels_df[['pid']] = df[['pid']]
    predict_labels_df.loc[:, predict_labels_df.columns != 'pid'] = predictions
    
    return predict_labels_df

## Train set

In [11]:
prediction_labels_df = predict(X_df)

print(prediction_labels_df.shape)
display(prediction_labels_df.head(15))
display(train_labels_df.head(15))

(18995, 16)


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,1,0.695221,0.507365,0.488484,0.493781,0.490149,0.542346,0.500991,0.523810,0.512515,0.482702,0.526559,17.086797,72.842049,99.005360,82.946919
1,10,0.519219,0.512416,0.519902,0.520110,0.521406,0.517092,0.514765,0.514627,0.518227,0.519680,0.524200,17.858473,97.174640,97.271412,79.397664
2,100,0.555262,0.525042,0.515245,0.515715,0.514883,0.509659,0.510431,0.510622,0.520945,0.496251,0.524931,16.932958,81.044724,96.373020,101.503872
3,1000,0.523703,0.475033,0.474955,0.475087,0.470402,0.661993,0.494186,0.614229,0.496172,0.750280,0.479087,17.149429,81.289889,97.497593,86.022054
4,10000,0.682595,0.517864,0.493538,0.492233,0.495173,0.505544,0.495490,0.594705,0.499491,0.496806,0.509827,16.047586,70.575571,97.349803,80.054041
5,10002,0.511515,0.525032,0.524968,0.525006,0.524940,0.573674,0.513972,0.489668,0.514181,0.500935,0.524936,19.596295,74.187292,96.970837,72.159673
6,10006,0.521616,0.520334,0.526124,0.526314,0.526650,0.516052,0.520112,0.521225,0.520889,0.506800,0.521775,19.000541,75.579660,96.353985,80.671766
7,10007,0.510015,0.509036,0.532366,0.533166,0.532779,0.515793,0.517120,0.517061,0.525002,0.516558,0.523576,19.957094,102.431610,95.941459,75.054190
8,10009,0.531709,0.508121,0.522625,0.522371,0.524960,0.520148,0.517932,0.525629,0.515180,0.493788,0.513654,20.763439,68.717797,96.552998,93.628288
9,1001,0.667379,0.525003,0.494744,0.496962,0.493399,0.632460,0.504983,0.474951,0.509592,0.502630,0.525094,14.978898,83.127920,99.074830,100.436828


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
2,100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
3,1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3
5,10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5,67.4,99.1,64.6
6,10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.2,84.9,96.8,90.5
7,10007,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,23.8,97.0,94.3,76.0
8,10009,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,18.9,72.0,95.3,91.5
9,1001,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14.9,83.3,99.9,117.3


In [12]:
prediction_labels_df.to_csv('data/prediction_train.csv', index=False, float_format='%.3f')

In [13]:
train_labels_df.to_csv('data/prediction_gold.csv', index=False, float_format='%.3f')

## Test set

In [14]:
prediction_test_labels_df = predict(X_test_df)

print(prediction_test_labels_df.shape)
display(prediction_test_labels_df.head(15))

(12664, 16)


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0,0.675854,0.666110,0.556290,0.550674,0.483014,0.624019,0.451693,0.532746,0.539265,0.570418,0.507824,19.894760,93.519089,100.549125,74.267266
1,10001,0.452897,0.518813,0.546825,0.550422,0.549925,0.532234,0.535124,0.566931,0.498260,0.515582,0.526621,27.781346,94.247281,92.775582,95.232599
2,10003,0.654469,0.514425,0.503757,0.495093,0.479878,0.446901,0.535030,0.476675,0.521824,0.498509,0.495090,26.108811,95.021495,97.501892,84.153224
3,10004,0.491074,0.532737,0.525611,0.532645,0.537352,0.529387,0.546160,0.559461,0.526127,0.535174,0.512230,24.168859,77.911129,93.657346,82.015460
4,10005,0.464566,0.477028,0.539215,0.542263,0.547421,0.526973,0.542423,0.517655,0.506066,0.517869,0.495416,31.701154,83.640013,94.068533,51.606157
5,10008,0.820557,0.536687,0.648631,0.652951,0.656138,0.688374,0.546207,0.464213,0.529776,0.517352,0.528427,29.608758,104.590588,96.315689,69.727029
6,10011,0.484596,0.534054,0.537904,0.540923,0.543780,0.555206,0.547149,0.536768,0.519656,0.503896,0.515425,25.817046,108.960148,98.779627,65.767689
7,10017,0.532405,0.497381,0.544552,0.553683,0.546164,0.565080,0.556480,0.555878,0.495481,0.530855,0.543355,33.159215,115.928835,97.705109,100.076163
8,10018,0.435835,0.477517,0.591772,0.595365,0.598403,0.555500,0.512794,0.564832,0.482440,0.567717,0.521452,28.212611,87.882865,92.343134,88.707217
9,10019,0.519606,0.465778,0.504697,0.498617,0.492838,0.474767,0.517153,0.629965,0.493131,0.509427,0.495737,26.212276,88.123114,94.284807,66.501460


In [15]:
prediction_test_labels_df.to_csv('data/prediction.csv', index=False, float_format='%.3f')